##### Copyright 2018 The TensorFlow Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License"); { display-mode: "form" }
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Deep Exponential Family with TFP

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://drive.google.com/file/d/1LiIZct-3Qk0yd7EkypjZABwcSPyeyhd0/view?usp=sharing"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href=""><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>
<br>
<br>
<br>


Original content [this Repository](https://github.com/blei-lab/edward) and [this paper](http://www.cs.toronto.edu/~lcharlin/papers/def_aistats.pdf), created by [the Blei Lab](http://www.cs.columbia.edu/~blei/)

Ported to Tensorflow Probability by Matthew McAteer ([`@MatthewMcAteer0`](https://twitter.com/MatthewMcAteer0)), with help from the TFP team at  Google ([`tfprobability@tensorflow.org`](mailto:tfprobability@tensorflow.org)).

---

>[Dependencies & Prerequisites](#scrollTo=2ZtWUjXYRXQi)

>[Introduction](#scrollTo=2ZtWUjXYRXQi)

>>[Data](#scrollTo=2ZtWUjXYRXQi)

>>[Model](#scrollTo=2ZtWUjXYRXQi)

>>[Inference](#scrollTo=2ZtWUjXYRXQi)

>>[Criticism](#scrollTo=2ZtWUjXYRXQi)

>[References](#scrollTo=2ZtWUjXYRXQi)

## Dependencies & Prerequisites

In [0]:
!pip3 install -q tfp-nightly
!pip3 install -q observations

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# import edward as ed
import numpy as np
import os
import tensorflow as tf

from datetime import datetime
# from edward.models import Gamma, Poisson, Normal, PointMass, TransformedDistribution
# from edward.util import Progbar
from observations import nips

In [0]:
def session_options(enable_gpu_ram_resizing=True, enable_xla=True):
    """
    Allowing the notebook to make use of GPUs if they're available.
    
    XLA (Accelerated Linear Algebra) is a domain-specific compiler for linear 
    algebra that optimizes TensorFlow computations.
    """
    config = tf.ConfigProto()
    config.log_device_placement = True
    if enable_gpu_ram_resizing:
        # `allow_growth=True` makes it possible to connect multiple colabs to your
        # GPU. Otherwise the colab malloc's all GPU ram.
        config.gpu_options.allow_growth = True
    if enable_xla:
        # Enable on XLA. https://www.tensorflow.org/performance/xla/.
        config.graph_options.optimizer_options.global_jit_level = (
            tf.OptimizerOptions.ON_1)
    return config


def reset_sess(config=None):
    """
    Convenience function to create the TF graph & session or reset them.
    """
    if config is None:
        config = session_options()
    global sess
    tf.reset_default_graph()
    try:
        sess.close()
    except:
        pass
    sess = tf.InteractiveSession(config=config)

    
def evaluate(tensors):
    """
    A "Universal" evaluate function for both running either Graph mode (default)
    or Eager mode (https://www.tensorflow.org/guide/eager) in Tensorflow.
    """
    if context.executing_eagerly():
        return (t.numpy() for t in tensprs)
    with tf.get_default_session() as sess:
        return sess.run(tensors)

reset_sess()


def strip_consts(graph_def, max_const_size=32):
  """
  Strip large constant values from graph_def.
  """
  strip_def = tf.GraphDef()
  for n0 in graph_def.node:
    n = strip_def.node.add()
    n.MergeFrom(n0)
    if n.op == 'Const':
      tensor = n.attr['value'].tensor
      size = len(tensor.tensor_content)
      if size > max_const_size:
        tensor.tensor_content = bytes("<stripped %d bytes>"%size, 'utf-8')
  return strip_def


def draw_graph(model, *args, **kwargs):
  """
  Visualize TensorFlow graph.
  """
  graph = tf.Graph()
  with graph.as_default():
    model(*args, **kwargs)
  graph_def = graph.as_graph_def()
  strip_def = strip_consts(graph_def, max_const_size=32)
  code = """
      <script>
        function load() {{
          document.getElementById("{id}").pbtxt = {data};
        }}
      </script>
      <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
      <div style="height:600px">
        <tf-graph-basic id="{id}"></tf-graph-basic>
      </div>
  """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

  iframe = """
      <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
  """.format(code.replace('"', '&quot;'))
  IPython.display.display(IPython.display.HTML(iframe))

## Introduction

Sparse Gamma deep exponential family (Ranganath et al., 2015).

### Abstract
We describe deep exponential families (DEFs), a class of latent variable models that are inspired by the hidden structures used in deep neural networks. DEFs capture a hierarchy of dependencies between latent variables, and are easily generalized to many settings through exponential families. We perform inference using recent “black box” variational inference techniques. We then evaluate various DEFs on text and combine multiple DEFs into a model for pairwise recommendation data. In an extensive study, we show going beyond one layer improves predictions for DEFs. We demonstrate that DEFs find interesting exploratory structure in large data sets, and give better predictive
performance than state-of-the-art models.

---

In this paper we develop deep exponential families
(DEFs), a flexible family of probability distributions
that reflect the intuitions behind deep unsupervised
feature learning. In a DEF, observations arise from
a cascade of layers of latent variables. Each layer’s
variables are drawn from an exponential family that is
governed by the inner product of the previous layer’s
variables and a set of weights.
As in deep unsupervised feature learning, a DEF represents
hidden patterns, from coarse to fine grained,
that compose with each other to form the observations.
DEFs also enjoy the advantages of probabilistic
modeling. Through their connection to exponential
families [7], they support many kinds of data. Overall
DEFs combine the powerful representations of deep
networks with the flexibility of graphical models.
Consider the problem of modeling documents. We can
represent a document as a vector of term counts modeled
with Poisson random variables [9]. In one type of
DEF, the rate of each term’s Poisson count is an inner
product of a layer of latent variables (one level up from
the terms) and a set of weights that are shared across
documents. Loosely, we can think of the latent layer
the observations as per-document “topic” activations,
each of which ignites a set of related terms via their inner
product with the weights. These latent topics are,
in turn, modeled in a similar way, conditioned on a
layer above of “super topics.” Just as the topics group
related terms, the super topics group related topics,
again via the inner product.
Figure 1 illustrates an example of a three level DEF
uncovered from a large set of articles in The New York
Times. (This style of model, though with different details,
has been previously studied in the topic modeling
literature [21].) Conditional on the word counts
of the articles, the DEF defines a posterior distribution
of the per-document cascades of latent variables
and the layers of weights. Here we have visualized
two third-layer topics which correspond to the concepts
of “Government” and “Politics”. We focus on
“Government” and notice that the model has discovered,
through its second-layer super-topics, the three
branches of government: judiciary ( left), legislative
(center) and executive (right).
This is just one example. In a DEF, the latent variables
can be from any exponential family: Bernoulli
latent variables recover the classical sigmoid belief
network [25]; Gamma latent variables give something
akin to deep version of nonnegative matrix factorization
[20]; Gaussian latent variables lead to the types of
models that have recently been explored in the context


## Our example

We apply it as a topic model on the collection of NIPS 2011 conference
papers. The loss function can sometimes erroneously output a negative value or NaN. This happens when the samples from the variational approximation
are numerically zero, which causes Gamma log probs to output inf. With default settings (in particular, with log normal variational
approximation), it takes ~62s per epoch on a Titan X (Pascal). The following results are on epoch 12.

```
10000/10000 [100%] ██████████████████████████████ Elapsed: 62s
Negative log-likelihood <= -1060649.607
Perplexity <= 0.205
Topic 0: let distribution set strategy distributions given learning information use property
Topic 1: functions problem risk function submodular cut level clustering sets performance
Topic 2: action value learning regret reward actions algorithm optimal state return
Topic 3: posterior stochastic approach information based using prior mean divergence since
Topic 4: player inference game propagation experts static query expert base variables
Topic 5: algorithm set loss weak algorithms optimal submodular online cost setting
Topic 6: sparse sparsity norm solution learning penalty greedy structure wise regularization
Topic 7: learning training linear kernel using coding accuracy performance dataset based
Topic 8: object categories image features examples classes images class objects visual
Topic 9: data manifold matrix points dimensional point low linear gradient optimization
```

A Gamma variational approximation produces worse results, which is
likely due to the high variance in stochastic gradients. It takes ~2
minutes per epoch on a Titan X (Pascal). Following results are on
epoch 12.
```
Negative log-likelihood <= 3738025.615
Perplexity <= 266.623
Topic 0: reasons posterior tion using similar tools university input computed refers
Topic 1: expected since much related rate defined optimization vector thus neurons
Topic 2: large linear given table shown true drop classification constraints current
Topic 3: proposed processing estimated better values gaussian form test true setting
Topic 4: see methods local several rate processing general vector enables section
Topic 5: thus case methods image dataset models different instead new respectively
Topic 6: based consider samples step object see kernel since problem training
Topic 7: approaches linear computing show gaussian data expected analysis well proof
Topic 8: fig point kernel bayesian solution applications results follows regression computer
Topic 9: conference optimization training pages maximum learning dataset performance state inference
```

In [0]:
# tf.flags.DEFINE_string("data_dir", default="~/data", help="")
# tf.flags.DEFINE_string("logdir", default="~/log/def/", help="")
# tf.flags.DEFINE_list("K", default=[100, 30, 15], help="Number of components per layer.")
# tf.flags.DEFINE_string("q", default="lognormal", help="Choice of q; 'lognormal' or 'gamma'.")
# tf.flags.DEFINE_float("shape", default=0.1, help="Gamma shape parameter.")
# tf.flags.DEFINE_float("lr", default=1e-4, help="Learning rate step-size.")
# 
# FLAGS = tf.flags.FLAGS
# FLAGS.data_dir = os.path.expanduser(FLAGS.data_dir)
# FLAGS.logdir = os.path.expanduser(FLAGS.logdir)
# timestamp = datetime.strftime(datetime.utcnow(), "%Y%m%d_%H%M%S")
# FLAGS.logdir += timestamp + '_' + '_'.join([str(ks) for ks in FLAGS.K]) + \
#     '_q_' + str(FLAGS.q) + '_lr_' + str(FLAGS.lr)


data_dir = "~/data"
logdir = "~/log/def/"
K =[100, 30, 15]  # Number of components per layer.")
q =" lognormal"   # Choice of q; 'lognormal' or 'gamma'.")
shape = 0.1       # Gamma shape parameter.")
lr = 1e-4         # Learning rate step-size.")

data_dir = os.path.expanduser(data_dir)
logdir = os.path.expanduser(.logdir)
timestamp = datetime.strftime(datetime.utcnow(), "%Y%m%d_%H%M%S")
logdir += timestamp + '_' + '_'.join([str(ks) for ks in K]) + '_q_' + str(q) + '_lr_' + str(lr)

In [0]:
def pointmass_q(shape, name=None):
    with tf.variable_scope(name, default_name="pointmass_q"):
        min_mean = 1e-3
        mean = tf.get_variable("mean", shape)
        rv = PointMass(tf.maximum(tf.nn.softplus(mean), min_mean))
        return rv

In [0]:
def gamma_q(shape, name=None):
    # Parameterize Gamma q's via shape and scale, with softplus unconstraints.
    with tf.variable_scope(name, default_name="gamma_q"):
        min_shape = 1e-3
        min_scale = 1e-5
        shape = tf.get_variable(
            "shape", shape,
            initializer=tf.random_normal_initializer(mean=0.5, stddev=0.1))
        scale = tf.get_variable(
            "scale", shape, initializer=tf.random_normal_initializer(stddev=0.1))
        rv = Gamma(tf.maximum(tf.nn.softplus(shape), min_shape),
                   tf.maximum(1.0 / tf.nn.softplus(scale), 1.0 / min_scale))
        return rv

In [0]:
def lognormal_q(shape, name=None):
    with tf.variable_scope(name, default_name="lognormal_q"):
        min_scale = 1e-5
        loc = tf.get_variable("loc", shape)
        scale = tf.get_variable(
            "scale", shape, initializer=tf.random_normal_initializer(stddev=0.1))
        rv = TransformedDistribution(
            distribution=Normal(loc, tf.maximum(tf.nn.softplus(scale), min_scale)),
            bijector=tf.contrib.distributions.bijectors.Exp())
        return rv

### Data

In [0]:
# ed.set_seed(42)

x_train, metadata = nips(data_dir)
documents = metadata['columns']
words = metadata['rows']

# Subset to documents in 2011 and words appearing in at least two
# documents and have a total word count of at least 10.
doc_idx = [i for i, document in enumerate(documents)
             if document.startswith('2011')]
documents = [documents[doc] for doc in doc_idx]
x_train = x_train[:, doc_idx]
word_idx = np.logical_and(np.sum(x_train != 0, 1) >= 2,
                            np.sum(x_train, 1) >= 10)
words = [word for word, idx in zip(words, word_idx) if idx]
x_train = x_train[word_idx, :]
x_train = x_train.T

N = x_train.shape[0]  # number of documents
D = x_train.shape[1]  # vocabulary size


### Model

In [0]:
W2 = Gamma(0.1, 0.3, sample_shape=[K[2], K[1]])
W1 = Gamma(0.1, 0.3, sample_shape=[K[1], K[0]])
W0 = Gamma(0.1, 0.3, sample_shape=[K[0], D])

z3 = Gamma(0.1, 0.1, sample_shape=[N, K[2]])
z2 = Gamma(shape, shape / tf.matmul(z3, W2))
z1 = Gamma(shape, shape / tf.matmul(z2, W1))
x = Poisson(tf.matmul(z1, W0))


### Inference

In [0]:
qW2 = pointmass_q(W2.shape)
qW1 = pointmass_q(W1.shape)
qW0 = pointmass_q(W0.shape)

if FLAGS.q == 'gamma':
    qz3 = gamma_q(z3.shape)
    qz2 = gamma_q(z2.shape)
    qz1 = gamma_q(z1.shape)
else:
    qz3 = lognormal_q(z3.shape)
    qz2 = lognormal_q(z2.shape)
    qz1 = lognormal_q(z1.shape)

# We apply variational EM with E-step over local variables
# and M-step to point estimate the global weight matrices.
inference_e = ed.KLqp({z1: qz1, z2: qz2, z3: qz3},
                        data={x: x_train, W0: qW0, W1: qW1, W2: qW2})
inference_m = ed.MAP({W0: qW0, W1: qW1, W2: qW2},
                       data={x: x_train, z1: qz1, z2: qz2, z3: qz3})

optimizer_e = tf.train.RMSPropOptimizer(FLAGS.lr)
optimizer_m = tf.train.RMSPropOptimizer(FLAGS.lr)
kwargs = {'optimizer': optimizer_e,
            'n_print': 100,
            'logdir': FLAGS.logdir,
            'log_timestamp': False}
if FLAGS.q == 'gamma':
    kwargs['n_samples'] = 30
inference_e.initialize(**kwargs)
inference_m.initialize(optimizer=optimizer_m)

sess = ed.get_session()
tf.global_variables_initializer().run()

n_epoch = 20
n_iter_per_epoch = 10000
for epoch in range(n_epoch):
    print("Epoch {}".format(epoch))
    nll = 0.0

    pbar = Progbar(n_iter_per_epoch)
    for t in range(1, n_iter_per_epoch + 1):
        pbar.update(t)
        info_dict_e = inference_e.update()
        info_dict_m = inference_m.update()
        nll += info_dict_e['loss']

    # Compute perplexity averaged over a number of training iterations.
    # The model's negative log-likelihood of data is upper bounded by
    # the variational objective.
    nll /= n_iter_per_epoch
    perplexity = np.exp(nll / np.sum(x_train))
    print("Negative log-likelihood <= {:0.3f}".format(nll))
    print("Perplexity <= {:0.3f}".format(perplexity))

    # Print top 10 words for first 10 topics.
    qW0_vals = sess.run(qW0)
    for k in range(10):
        top_words_idx = qW0_vals[k, :].argsort()[-10:][::-1]
        top_words = " ".join([words[i] for i in top_words_idx])
        print("Topic {}: {}".format(k, top_words))

In [0]:
# Visualizing the graph we've constructed
# draw_graph(linear_mixed_effects_model, features_train)

## Reference

1. (Ranganath et al., 2015). [Deep Exponential Families](http://www.cs.toronto.edu/~lcharlin/papers/def_aistats.pdf)

[1] Y. Bengio, A. Courville, and P. Vincent. Representation learning: A review and new perspectives. Pattern Analysis and Machine Intelligence, 35(8), 2013.

[2] C. Bishop. Pattern Recognition and Machine Learning. Springer New York., 2006.

[3] D. Blei and J. Lafferty. Dynamic topic models. In International Conference on Machine Learning, 2006.

[4] D. Blei and J. Lafferty. A correlated topic model of Science. Annals of Applied Stat., 1(1):17–35, 2007.

[5] D. Blei, T. Griffiths, M. Jordan, and J. Tenenbaum. Hierarchical topic models and the nested Chinese restaurant process. In NIPS, 2003.

[6] D. Blei, A. Ng, and M. Jordan. Latent Dirichlet allocation. Journal of Machine Learning Research, 3: 993–1022, January 2003.

[7] L. Brown. Fundamentals of Statistical Exponential Families. Institute of Mathematical Statistics, 1986.

[8] W. Buntine and A. Jakulin. Applying discrete PCA in data analysis. In Proceedings of the 20th Conference on Uncertainty in Artificial Intelligence, pages 59–66. AUAI Press, 2004.

[9] J. Canny. GaP: A factor model for discrete data. In Proceedings of the 27th Annual International ACM SIGIR Conference on Research and Development in Information Retrieval, 2004.

[10] A. Gelman and J. Hill. Data Analysis Using Regression and Multilevel/Hierarchical Models. Cambridge Univ. Press, 2007.

[11] Andrew Gelman, John B Carlin, Hal S Stern, David B Dunson, Aki Vehtari, and Donald B Rubin. Bayesian data analysis. CRC press, 2013.

[12] I. Goodfellow, A. Courville, and Y. Bengio. Largescale feature learning with spike-and-slab sparse coding. In International Conference on Machine Learning (ICML), 2012.

[13] P. Gopalan, J. Hoffman, and D. Blei. Scalable recommendation with poisson factorization. arXiv, (1311.1704), 2013.

[14] Daniel Hern´andez-Lobato, Jos´e Miguel Hern´andezLobato, and Pierre Dupont. Generalized spike-andslab priors for bayesian group feature selection using expectation propagation. The Journal of Machine Learning Research, 14(1):1891–1945, 2013.

[15] G. Hinton, S. Osindero, and Y. Teh. A fast learning algorithm for deep belief nets. Neural Comput., 18(7): 1527–1554, July 2006. ISSN 0899-7667.

[16] H. Ishwaran and S. Rao. Spike and slab variable selection: Frequentist and Bayesian strategies. The Annals of Statistics, 33(2):730–773, 2005.

[17] Kalervo J¨arvelin and Jaana Kek¨al¨ainen. Ir evaluation methods for retrieving highly relevant documents. In In International ACM SIGIR Conference on Research and Development in Information Retrieval, SIGIR ’00, pages 41–48. ACM, 2000. ISBN 1-58113-226-3.

[18] M. Jordan, Z. Ghahramani, T. Jaakkola, and L. Saul. Introduction to variational methods for graphical models. Machine Learning, 37:183–233, 1999.

[19] Hugo Larochelle and Stanislas Lauly. A neural autoregressive topic model. In Neural Information Processing Systems, 2012.

[20] D. Lee and H. Seung. Learning the parts of objects by non-negative matrix factorization. Nature, 401(6755): 788–791, October 1999.

[21] W. Li and A. McCallum. Pachinko allocation:DAGstuctured mixture models of topic correlations. In ICML, 2006.

[22] Benjamin Marlin. Collaborative filtering: A machine learning perspective. Technical report, University of Toronto, 2004.

[23] A. Mnih and K. Gregor. Neural variational inference and learning in belief networks. In ICML, 2014.

[24] S. Mohamed, K. Heller, and Z. Ghahramani. Bayesian exponential family PCA. In NIPS, 2008.

[25] R. Neal. Learning stochastic feedforward networks. Tech. Rep. CRG-TR-90-7: Department of Computer Science, University of Toronto, 1990.

[26] J. A. Nelder and R. W. M. Wedderburn. Generalized linear models. Journal of the Royal Statistical Society. Series A (General), 135:370–384, 1972.

[27] R. Ranganath, S. Gerrish, and D. Blei. Black box variational inference. In International Conference on Artifical Intelligence and Statistics, 2014.

[28] D. Rezende, S. Mohamed, and D. Wierstra. Stochastic backpropagation and approximate inference in deep generative models. ArXiv e-prints, January 2014.

[29] H. Robbins and S. Monro. A stochastic approximation method. The Annals of Mathematical Statistics, 22(3): pp. 400–407, 1951.

[30] R. Salakhutdinov and G. Hinton. Deep boltzmann machines. In AISTATS, pages 448–455, 2009.

[31] R. Salakhutdinov and A. Mnih. Bayesian probabilistic matrix factorization using Markov chain Monte Carlo. In International Conference on Machine learning, 2008.

[32] R. Salakhutdinov and A. Mnih. Probabilistic matrix factorization. In Neural Information Processing Systems, 2008.

[33] Ruslan Salakhutdinov and Geoffrey Hinton. Replicated softmax: an undirected topic model. In Y. Bengio, D. Schuurmans, J. Lafferty, C. K. I. Williams, and A. Culotta, editors, Advances in Neural Information Processing Systems 22, pages 1607–1614. 2009.

[34] T. Salimans and D. Knowles. Fixed-form variational posterior approximation through stochastic linear regression. Bayesian Analysis, 8(4):837–882, 2013
.
[35] L. Saul, T. Jaakkola, and M. Jordan. Mean field theory for sigmoid belief networks. Journal of Artificial Intelligence Research, 4:61–76, 1996.

In [0]:
from IPython.core.display import HTML
def css_styling():
    styles = open("../styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

#  "#F15854",  // red
#  "#5DA5DA",  // blue
#  "#FAA43A",  // orange
#  "#60BD68",  // green
#  "#F17CB0",  // pink
#  "#B2912F",  // brown
#  "#B276B2",  // purple
#  "#DECF3F",  // yellow
#  "#4D4D4D",  // gray